In [33]:
from zipfile import ZipFile
import json
import pandas as pd

In [34]:
from reportlab.pdfgen import canvas
from reportlab.platypus import *
from reportlab.lib import colors
from reportlab.lib.pagesizes import A4
import pandas as pd
import numpy as np

In [35]:
shifts = '../shift-definitions/shift-definitions_23-12-2023_16-39-07.zip'

In [36]:
with ZipFile(shifts, 'r') as zipfile:
    name_list = zipfile.namelist()
    with zipfile.open('G1_GVE_22-12-2023_08-29-53.json') as shift:
        #print(shift)
        shift_task_list = json.load(shift)
        #print(shift_task_list)

In [37]:
name_list

['A4_GVE_05-12-2023_15-10-14.json',
 'ZUE-Test_15-11-2023_16-11-11.json',
 'gve-00-12_07-11-2023_17-07-14.json',
 'gve-12-00_07-11-2023_17-13-47.json',
 'A1.ZUE.test_11-12-2023_14-37-34.json',
 'G3_GVE_05-12-2023_15-02-25.json',
 'G4_GVE_05-12-2023_15-03-57.json',
 'O1_GVE_06-12-2023_08-40-35.json',
 'A2_GVE_05-12-2023_15-09-37.json',
 'G1_LOM_15-12-2023_07-37-05.json',
 'G2_LOM_15-12-2023_08-01-20.json',
 'G1_ZUE_11-12-2023_16-44-03.json',
 'G1_GVE_22-12-2023_08-29-53.json']

In [38]:
tasks = pd.json_normalize(shift_task_list['taskDefinitions'])

In [39]:
tasks.set_index('id', inplace=True)

In [40]:
tasks.columns.values

array(['version', 'name', 'description', 'unit', 'intendedStartTime',
       'allowedStartTime', 'zoneId', 'duration', 'onDemand', 'commands',
       'auditData.createdBy', 'auditData.createdTime',
       'auditData.lastModifiedBy', 'auditData.lastModifiedTime',
       'schedule.startDate', 'schedule.endDate', 'schedule.expression'],
      dtype=object)

In [41]:
tasks_cleaned = tasks.drop(['version', \
                            'name',\
                            'unit', \
                            'onDemand',\
                            'allowedStartTime',\
                            'duration', \
                            'commands', \
                            'schedule.startDate', \
                            'schedule.endDate', \
                            'schedule.expression', \
                            'auditData.createdBy', \
                            'auditData.createdTime', \
                            'auditData.lastModifiedBy', \
                            'auditData.lastModifiedTime'], axis=1)

In [42]:
tasks_cleaned.columns.values

array(['description', 'intendedStartTime', 'zoneId'], dtype=object)

In [43]:
tasks_cleaned

,description,intendedStartTime,zoneId
id,,,
a22c49bb-2c99-4275-a4e7-057033391710,Rappels : Vent et Neige,06:15:00,Europe/Zurich
a06e44df-d48c-479c-a48e-2971187792af,"Consulter APW blog, main courante, mise à jour...",06:15:00,Europe/Zurich
c6c99e22-dc61-461a-b9fe-3788f5074999,Bulletin Radio RTS 07h (envoi avant 06h45),06:20:00,Europe/Zurich
35e24943-bef5-44ba-93c7-ab4d11b7c797,Vérifier présence tour Clim à l'horaire (sinon...,06:25:00,Europe/Zurich
66f5dd0c-8860-4c6c-82cf-ef0885b754f5,Mise a jour bulletin général,06:30:00,Europe/Zurich
c9262502-abb3-48d6-b9c8-c19ff1964775,Evaluer le besoin d'un tour R et appeler à 7h ...,06:50:00,Europe/Zurich
87ecef1d-70b8-4d07-8c6c-12dd927f5ac6,Probabilités d'orage pour le briefing Skyguide,07:00:00,Europe/Zurich
f619906a-a02a-4d5a-b287-82bb441e44dc,Proposition nouvelle neige SLF -> 3 périodes c...,09:20:00,Europe/Zurich


In [59]:
tasks_ordered = tasks_cleaned.loc[:,['intendedStartTime','zoneId','description']]

In [60]:
doc = SimpleDocTemplate("table.pdf", pagesize=A4)

In [61]:
t1 = Table(np.array(tasks_ordered).tolist())
t1

Table(
 rowHeights=[None, None, None, None, None, None, None, None],
 colWidths=[None, None, None],
[['06:15:00', 'Europe/Zurich', 'Rappels : Vent et Neige'],
   ['06:15:00',
    'Europe/Zurich',
    'Consulter APW blog, main courante, mise à jour produits et mails du compte '
    'cmg'],
   ['06:20:00', 'Europe/Zurich', 'Bulletin Radio RTS 07h (envoi avant 06h45)'],
   ['06:25:00',
    'Europe/Zurich',
    "Vérifier présence tour Clim à l'horaire (sinon, appels clim pris en salle "
    'de prévision)'],
   ['06:30:00', 'Europe/Zurich', 'Mise a jour bulletin général'],
   ['06:50:00',
    'Europe/Zurich',
    "Evaluer le besoin d'un tour R et appeler à 7h au besoin"],
   ['07:00:00',
    'Europe/Zurich',
    "Probabilités d'orage pour le briefing Skyguide"],
   ['09:20:00',
    'Europe/Zurich',
    'Proposition nouvelle neige SLF -> 3 périodes consécutives de 24 heures']]
) # end table

In [62]:
doc = SimpleDocTemplate("table.pdf", pagesize=A4)

In [63]:
element = []
element.append(t1)
doc.build(element)